In [121]:
# Cell 1: Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import math

In [122]:
# Cell 2: Create directories
os.makedirs('../results/30_split_training_30e/plots', exist_ok=True)
os.makedirs('../results/30_split_training_30e/plots/individual_clients', exist_ok=True)

In [123]:
# Cell 3: Configuration - CHANGE NUM_CLIENTS HERE
# ============================================
# FEDERATED LEARNING PARAMETERS
# ============================================
NUM_CLIENTS = 30     # <-- CHANGE THIS TO 5, 10, 20, etc.
NUM_ROUNDS = 2      # Number of communication rounds
LOCAL_EPOCHS = 30   # Same as model1.ipynb
BATCH_SIZE = 64     # Same as model1.ipynb

print("=" * 60)
print("FEDERATED LEARNING CONFIGURATION")
print("=" * 60)
print(f"Number of Clients: {NUM_CLIENTS}")
print(f"Communication Rounds: {NUM_ROUNDS}")
print(f"Local Epochs per Round: {LOCAL_EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print("=" * 60 + "\n")

FEDERATED LEARNING CONFIGURATION
Number of Clients: 30
Communication Rounds: 2
Local Epochs per Round: 30
Batch Size: 64



In [124]:
# Cell 4: Load and normalize client data
# ============================================
# LOAD AND NORMALIZE CLIENT DATA
# ============================================
print("=" * 60)
print("LOADING AND NORMALIZING CLIENT DATA")
print("=" * 60)

client_data = []
for i in range(1, NUM_CLIENTS + 1):  # Dynamic based on NUM_CLIENTS
    data_path = f'../data/30_split_data/mnist_part{i}.npz'
    
    # Load .npz file
    data = np.load(data_path)
    
    # Get the data
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']
    
    # Normalize (same as model1.ipynb)
    x_train = x_train / 255
    x_test = x_test / 255
    
    # Flatten (same as model1.ipynb)
    x_train_flatten = x_train.reshape(len(x_train), 28*28)
    x_test_flatten = x_test.reshape(len(x_test), 28*28)
    
    # Store normalized and flattened data
    normalized_data = {
        'x_train': x_train_flatten,
        'y_train': y_train,
        'x_test': x_test_flatten,
        'y_test': y_test
    }
    
    client_data.append(normalized_data)
    print(f"✓ Client {i} - Train: {len(x_train_flatten)}, Test: {len(x_test_flatten)}")

print(f"\n✓ Total clients loaded: {len(client_data)}\n")

LOADING AND NORMALIZING CLIENT DATA
✓ Client 1 - Train: 2000, Test: 334
✓ Client 2 - Train: 2000, Test: 334
✓ Client 3 - Train: 2000, Test: 334
✓ Client 4 - Train: 2000, Test: 334
✓ Client 5 - Train: 2000, Test: 334
✓ Client 6 - Train: 2000, Test: 334
✓ Client 7 - Train: 2000, Test: 334
✓ Client 8 - Train: 2000, Test: 334
✓ Client 9 - Train: 2000, Test: 334
✓ Client 10 - Train: 2000, Test: 334
✓ Client 11 - Train: 1999, Test: 334
✓ Client 12 - Train: 1999, Test: 334
✓ Client 13 - Train: 1999, Test: 334
✓ Client 14 - Train: 1999, Test: 334
✓ Client 15 - Train: 1999, Test: 334
✓ Client 16 - Train: 1999, Test: 334
✓ Client 17 - Train: 1999, Test: 334
✓ Client 18 - Train: 1999, Test: 334
✓ Client 19 - Train: 1999, Test: 334
✓ Client 20 - Train: 1999, Test: 334
✓ Client 21 - Train: 1999, Test: 334
✓ Client 22 - Train: 1999, Test: 334
✓ Client 23 - Train: 1999, Test: 334
✓ Client 24 - Train: 1999, Test: 334
✓ Client 25 - Train: 1999, Test: 334
✓ Client 26 - Train: 1999, Test: 334
✓ Client 27

In [125]:
# Cell 5: Define model architecture
# ============================================
# DEFINE MODEL ARCHITECTURE (EXACT COPY FROM model1.ipynb)
# ============================================
def create_model():
    """Exact same model as model1.ipynb"""
    model = keras.Sequential([
        # First hidden layer - Feature extraction
        keras.layers.Dense(128, input_shape=(784,), activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.3),
        
        # Second hidden layer - Pattern recognition
        keras.layers.Dense(64, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Third hidden layer - Feature refinement
        keras.layers.Dense(32, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Output layer - Classification
        keras.layers.Dense(10, activation="softmax")
    ])

    # Compile with optimized parameters (same as model1.ipynb)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [126]:
# Cell 6: Federated averaging function
# ============================================
# FEDERATED AVERAGING FUNCTION
# ============================================
def average_weights(weights_list):
    """Average weights from all clients"""
    averaged_weights = []
    for weights_tuple in zip(*weights_list):
        averaged_weights.append(np.mean(weights_tuple, axis=0))
    return averaged_weights

In [127]:
# Cell 7: Initialize global model
# ============================================
# INITIALIZE GLOBAL MODEL
# ============================================
print("Initializing global model...")
global_model = create_model()
global_model.summary()
global_weights = global_model.get_weights()
print("\n✓ Global model initialized\n")

# Track accuracy and history for each client
client_histories = {f'client_{i}': [] for i in range(NUM_CLIENTS)}
client_test_accuracies = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Store training histories for plotting
all_training_histories = []

Initializing global model...


Model: "sequential_692"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2768 (Dense)              │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2076        │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2076 (Dropout)          │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2769 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2077        │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2077 (Dropout)          │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2770 (Dense)              │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2078        │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2078 (Dropout)          │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2771 (Dense)              │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,042 (437.66 KB)

 Trainable params: 111,594 (435.91 KB)

 Non-trainable params: 448 (1.75 KB)


✓ Global model initialized



In [128]:
# Cell 8: Federated learning training loop
# ============================================
# FEDERATED LEARNING TRAINING LOOP
# ============================================
print("=" * 60)
print("STARTING FEDERATED LEARNING")
print("=" * 60 + "\n")

for round_num in range(NUM_ROUNDS):
    print(f"\n{'='*70}")
    print(f"COMMUNICATION ROUND {round_num + 1}/{NUM_ROUNDS}")
    print(f"{'='*70}\n")
    
    local_weights = []
    round_histories = []
    
    # Train each client
    for client_id in range(NUM_CLIENTS):
        print(f"\n{'─'*70}")
        print(f"[Client {client_id + 1}] Starting local training...")
        print(f"{'─'*70}")
        
        # Create fresh model and load global weights
        client_model = create_model()
        client_model.set_weights(global_weights)
        
        # Get client's normalized data
        x_train = client_data[client_id]['x_train']
        y_train = client_data[client_id]['y_train']
        x_test = client_data[client_id]['x_test']
        y_test = client_data[client_id]['y_test']
        
        # Train locally (same as model1.ipynb)
        history = client_model.fit(
            x_train, y_train,
            epochs=LOCAL_EPOCHS,
            batch_size=BATCH_SIZE,
            validation_split=0.1,
            verbose=1
        )
        
        # Store history for this round
        round_histories.append({
            'client_id': client_id,
            'round': round_num,
            'history': history
        })
        
        # Evaluate on test set
        print(f"\n[Client {client_id + 1}] Evaluating on test set...")
        loss, accuracy = client_model.evaluate(x_test, y_test, verbose=0)
        
        print(f"\n{'─'*70}")
        print(f"🎯 Client {client_id + 1} Test Accuracy: {accuracy * 100:.2f}%")
        print(f"{'─'*70}\n")
        
        # Store test accuracy
        client_test_accuracies[f'client_{client_id}'].append(accuracy)
        
        # Store weights for averaging
        local_weights.append(client_model.get_weights())
    
    # Store all histories for this round
    all_training_histories.append(round_histories)
    
    # Federated Averaging (Weight Exchange)
    print(f"\n{'🔄'*35}")
    print("WEIGHT EXCHANGE - FEDERATED AVERAGING")
    print(f"{'🔄'*35}")
    global_weights = average_weights(local_weights)
    global_model.set_weights(global_weights)
    print("✓ Global weights updated")
    print(f"{'🔄'*35}\n")
    

print("\n" + "="*70)
print("FEDERATED LEARNING COMPLETE!")
print("="*70 + "\n")

STARTING FEDERATED LEARNING


COMMUNICATION ROUND 1/2


──────────────────────────────────────────────────────────────────────
[Client 1] Starting local training...
──────────────────────────────────────────────────────────────────────
Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2983 - loss: 2.1559 - val_accuracy: 0.1250 - val_loss: 2.2940
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5183 - loss: 1.4556 - val_accuracy: 0.1250 - val_loss: 2.2887
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6278 - loss: 1.2068 - val_accuracy: 0.1250 - val_loss: 2.2872
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6978 - loss: 1.0123 - val_accuracy: 0.1250 - val_loss: 2.2836
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7156 - loss: 0.9549 - val_accuracy: 0.1250 - val_loss: 2.2736
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7500 - loss: 0.8922 - val_accuracy: 0.1250 - val_loss: 2.2575
Epoch 7

In [129]:
# Cell 9: Display final results
# ============================================
# DISPLAY FINAL RESULTS
# ============================================
print("FINAL RESULTS:")
print("-" * 70)
for client_id in range(NUM_CLIENTS):
    final_acc = client_test_accuracies[f'client_{client_id}'][-1]
    print(f"Client {client_id + 1} Final Accuracy: {final_acc*100:.2f}%")
print("-" * 70 + "\n")

FINAL RESULTS:
----------------------------------------------------------------------
Client 1 Final Accuracy: 88.92%
Client 2 Final Accuracy: 91.02%
Client 3 Final Accuracy: 88.92%
Client 4 Final Accuracy: 85.03%
Client 5 Final Accuracy: 89.22%
Client 6 Final Accuracy: 91.02%
Client 7 Final Accuracy: 89.22%
Client 8 Final Accuracy: 89.22%
Client 9 Final Accuracy: 87.72%
Client 10 Final Accuracy: 88.92%
Client 11 Final Accuracy: 88.92%
Client 12 Final Accuracy: 88.02%
Client 13 Final Accuracy: 91.62%
Client 14 Final Accuracy: 89.22%
Client 15 Final Accuracy: 91.62%
Client 16 Final Accuracy: 86.83%
Client 17 Final Accuracy: 88.92%
Client 18 Final Accuracy: 86.83%
Client 19 Final Accuracy: 81.74%
Client 20 Final Accuracy: 90.42%
Client 21 Final Accuracy: 88.92%
Client 22 Final Accuracy: 89.22%
Client 23 Final Accuracy: 85.33%
Client 24 Final Accuracy: 91.62%
Client 25 Final Accuracy: 90.12%
Client 26 Final Accuracy: 90.12%
Client 27 Final Accuracy: 88.92%
Client 28 Final Accuracy: 90.72%

In [130]:
# Cell 10: Generate plots - INDIVIDUAL CLIENT GRAPHS
# ============================================
# GENERATE PLOTS - EACH CLIENT'S TEST ACCURACY VS COMMUNICATION ROUNDS
# ============================================
print("=" * 70)
print("GENERATING PLOTS")
print("=" * 70)

# ────────────────────────────────────────────
# Plot 1: Individual graphs for each client (SEPARATE GRAPHS - 5 GRAPHS)
# ────────────────────────────────────────────
print("\n1. Creating individual client test accuracy plots...")

for client_id in range(NUM_CLIENTS):
    plt.figure(figsize=(10, 6))
    
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=3, markersize=10, 
             color='blue', label=f'Client {client_id + 1}')
    
    # Add accuracy values on points
    for i, acc in enumerate(accuracies):
        plt.text(i + 1, acc + 0.3, f'{acc:.2f}%', 
                ha='center', fontsize=10, fontweight='bold')
    
    plt.title(f'Client {client_id + 1}: Test Accuracy vs Communication Rounds', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Communication Round', fontsize=13)
    plt.ylabel('Test Accuracy (%)', fontsize=13)
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, NUM_ROUNDS + 1))
    plt.ylim([85, 100])
    
    save_path = f'../results/30_split_training_30e/plots/individual_clients/client_{client_id + 1}_accuracy_vs_rounds.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"   ✓ Saved: client_{client_id + 1}_accuracy_vs_rounds.png")
    plt.close()

# ────────────────────────────────────────────
# Plot 2: All clients in one grid (COMBINED VIEW - 5 SUBPLOTS)
# ────────────────────────────────────────────
print("\n2. Creating combined grid of all clients...")

cols = min(3, NUM_CLIENTS)
rows = math.ceil(NUM_CLIENTS / cols)

fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))

# Handle single row/column case
if NUM_CLIENTS == 1:
    axes = np.array([[axes]])
elif rows == 1:
    axes = axes.reshape(1, -1)
elif cols == 1:
    axes = axes.reshape(-1, 1)

fig.suptitle('Test Accuracy vs Communication Rounds (Each Client\'s Own Test Data)', 
             fontsize=18, fontweight='bold')

colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

for client_id in range(NUM_CLIENTS):
    row = client_id // cols
    col = client_id % cols
    
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    
    axes[row, col].plot(range(1, NUM_ROUNDS + 1), accuracies, 
                        marker='o', linewidth=2.5, markersize=8, 
                        color=colors[client_id])
    
    # Add accuracy values on points
    for i, acc in enumerate(accuracies):
        axes[row, col].text(i + 1, acc + 0.3, f'{acc:.1f}%', 
                           ha='center', fontsize=8, fontweight='bold')
    
    axes[row, col].set_title(f'Client {client_id + 1}', fontsize=14, fontweight='bold')
    axes[row, col].set_xlabel('Communication Round', fontsize=11)
    axes[row, col].set_ylabel('Test Accuracy (%)', fontsize=11)
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].set_xticks(range(1, NUM_ROUNDS + 1))
    axes[row, col].set_ylim([85, 100])
    
    # Add start and end accuracy text box
    start_acc = accuracies[0]
    end_acc = accuracies[-1]
    improvement = end_acc - start_acc
    axes[row, col].text(0.02, 0.02, 
                       f'Start: {start_acc:.1f}%\nEnd: {end_acc:.1f}%\nΔ: {improvement:+.1f}%',
                       transform=axes[row, col].transAxes,
                       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
                       fontsize=8, verticalalignment='bottom')

# Hide empty subplots
total_subplots = rows * cols
for idx in range(NUM_CLIENTS, total_subplots):
    row = idx // cols
    col = idx % cols
    axes[row, col].axis('off')

plt.tight_layout()
save_path = '../results/30_split_training_30e/plots/all_clients_accuracy_vs_rounds_grid.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: all_clients_accuracy_vs_rounds_grid.png")
plt.close()

# ────────────────────────────────────────────
# Plot 3: All clients on same graph (OVERLAY)
# ────────────────────────────────────────────
print("\n3. Creating overlay plot of all clients...")

plt.figure(figsize=(12, 7))

colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

for client_id in range(NUM_CLIENTS):
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=2.5, markersize=8, 
             color=colors[client_id],
             label=f'Client {client_id + 1}')

plt.title('Test Accuracy vs Communication Rounds (All Clients)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = '../results/30_split_training_30e/plots/all_clients_accuracy_overlay.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: all_clients_accuracy_overlay.png")
plt.close()

# ────────────────────────────────────────────
# Plot 4: Average of all clients with standard deviation
# ────────────────────────────────────────────
print("\n4. Creating average client accuracy plot...")

plt.figure(figsize=(12, 7))

# Calculate average accuracy per round
avg_accuracies = []
std_accuracies = []
for round_num in range(NUM_ROUNDS):
    round_accs = [client_test_accuracies[f'client_{i}'][round_num] * 100 for i in range(NUM_CLIENTS)]
    avg_accuracies.append(np.mean(round_accs))
    std_accuracies.append(np.std(round_accs))

plt.plot(range(1, NUM_ROUNDS + 1), avg_accuracies, 
         marker='s', linewidth=3, markersize=10, 
         color='red', label=f'Average ({NUM_CLIENTS} clients)')

# Add error bars for standard deviation
plt.fill_between(range(1, NUM_ROUNDS + 1), 
                 np.array(avg_accuracies) - np.array(std_accuracies),
                 np.array(avg_accuracies) + np.array(std_accuracies),
                 alpha=0.2, color='red')

# Add accuracy values on points
for i, acc in enumerate(avg_accuracies):
    plt.text(i + 1, acc + 0.5, f'{acc:.2f}%', 
            ha='center', fontsize=10, fontweight='bold')

plt.title('Average Test Accuracy vs Communication Rounds', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = '../results/30_split_training_30e/plots/average_accuracy_vs_rounds.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: average_accuracy_vs_rounds.png")
plt.close()

print("\n" + "="*70)
print("ALL PLOTS GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nConfiguration: {NUM_CLIENTS} clients, {NUM_ROUNDS} rounds")
print("\nPlots saved in:")
print(f"  - ../results/30_split_training_30e/plots/individual_clients/ ({NUM_CLIENTS} individual plots)")
print("  - ../results/30_split_training_30e/plots/all_clients_accuracy_vs_rounds_grid.png")
print("  - ../results/30_split_training_30e/plots/all_clients_accuracy_overlay.png")
print("  - ../results/30_split_training_30e/plots/average_accuracy_vs_rounds.png")
print("="*70)

GENERATING PLOTS

1. Creating individual client test accuracy plots...
   ✓ Saved: client_1_accuracy_vs_rounds.png
   ✓ Saved: client_2_accuracy_vs_rounds.png
   ✓ Saved: client_3_accuracy_vs_rounds.png
   ✓ Saved: client_4_accuracy_vs_rounds.png
   ✓ Saved: client_5_accuracy_vs_rounds.png
   ✓ Saved: client_6_accuracy_vs_rounds.png
   ✓ Saved: client_7_accuracy_vs_rounds.png
   ✓ Saved: client_8_accuracy_vs_rounds.png
   ✓ Saved: client_9_accuracy_vs_rounds.png
   ✓ Saved: client_10_accuracy_vs_rounds.png
   ✓ Saved: client_11_accuracy_vs_rounds.png
   ✓ Saved: client_12_accuracy_vs_rounds.png
   ✓ Saved: client_13_accuracy_vs_rounds.png
   ✓ Saved: client_14_accuracy_vs_rounds.png
   ✓ Saved: client_15_accuracy_vs_rounds.png
   ✓ Saved: client_16_accuracy_vs_rounds.png
   ✓ Saved: client_17_accuracy_vs_rounds.png
   ✓ Saved: client_18_accuracy_vs_rounds.png
   ✓ Saved: client_19_accuracy_vs_rounds.png
   ✓ Saved: client_20_accuracy_vs_rounds.png
   ✓ Saved: client_21_accuracy_vs_roun

In [131]:
# Cell 11: Save final model
# ============================================
# SAVE FINAL GLOBAL MODEL
# ============================================
model_save_path = '../results/30_split_training_30e/federated_global_model_final.h5'
global_model.save(model_save_path)
print(f"\n✓ Final global model saved to: {model_save_path}")

# Save accuracy data as numpy file for future reference
accuracy_data = {
    # REMOVED: 'global_accuracy_history': global_accuracy_history,
    'client_test_accuracies': client_test_accuracies,
    'num_clients': NUM_CLIENTS,
    'num_rounds': NUM_ROUNDS
}
np.save('../results/30_split_training_30e/accuracy_data.npy', accuracy_data)
print(f"✓ Accuracy data saved to: ../results/30_split_training_30e/accuracy_data.npy")

print("\n✅ FEDERATED LEARNING SIMULATION COMPLETE!")
print(f"   Clients: {NUM_CLIENTS}")
print(f"   Rounds: {NUM_ROUNDS}")
# REMOVED: print(f"   Final Global Accuracy: {global_accuracy_history[-1]*100:.2f}%")


✓ Final global model saved to: ../results/30_split_training_30e/federated_global_model_final.h5
✓ Accuracy data saved to: ../results/30_split_training_30e/accuracy_data.npy

✅ FEDERATED LEARNING SIMULATION COMPLETE!
   Clients: 30
   Rounds: 2
